In [1]:

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from distutils.version import StrictVersion
import pytesseract
import shutil
import tensorflow as tf
import re
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import inflect
import uuid

from mining_pages_utils.image_ocr_utils import load_page, cut_image, ocr_pre_processing, ocr_post_processing_pageid, cut_image_savetemp, cut_image_figid, ocr_post_processing_figid
from mining_pages_utils.dataframe_utils import get_page_labelmap_as_df, get_figid_labelmap_as_df, extract_page_detections, extract_page_detections_new,unfold_pagedetections, page_detections_toframe, extract_detections_figureidv2,humanreadID
from mining_pages_utils.dataframe_utils import filter_best_page_detections, select_pdfpages, choose_pageid, filter_best_vesselprofile_detections, merge_info,  provide_pagelist, provide_pdf_path, get_pubs_and_configs, pdf_to_image, handleduplicate_humanreadID
from mining_pages_utils.json_utils import create_find_JSONL, create_constructivisttype_JSONL, create_normativtype_JSONL, create_drawing_JSONL, create_catalog_JSONL, create_trench_JSONL
from mining_pages_utils.tensorflow_utils import create_tf_example_new, create_tf_figid, run_inference_for_page_series, run_inference_for_figure_series, build_detectfn, Df2TFrecord, split


if StrictVersion(tf.version.VERSION) < StrictVersion('2.3.0'):
    raise ImportError(
        'Please upgrade your TensorFlow installation to v1.9.* or later!')

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

INPUTDIRECTORY = '/home/images/apply' 
GRAPH = '/frozen_inference_graph.pb'
LABELS = '/label_map.pbtxt'
PAGE_MODEL = '/home/models/faster_rcnn_resnet101_v1_1024x1024_coco17_miningpagesv9'
SAVEDMODEL = '/saved_model'
FIGID_MODEL = '/home/models/faster_rcnn_resnet101_v1_1024x1024_coco17_miningfiguresv3'
SEG_MODEL = '/home/models/shape_segmentation/train_colab_20200610.h5'
OUTPATH = '/home/images/OUTPUT/'
VESSELLPATH = OUTPATH + 'vesselprofiles/'
SEGMENTPATH = OUTPATH + 'segmented_profiles/'
CSVOUT = OUTPATH + 'mining_pages_allinfo.csv'
CLEANCSVOUT = OUTPATH + 'mining_pages_clean.csv'









Segmentation Models: using `tf.keras` framework.


In [2]:
publist = get_pubs_and_configs(INPUTDIRECTORY)
pdflist = provide_pdf_path(publist)
pdflistv2 = pdflist.apply(pdf_to_image, axis=1)
pagelist = provide_pagelist(pdflistv2)
pagelist = select_pdfpages(pagelist)





<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class 'int'>
(40, 46)
(76, 77)
<class '

In [3]:

for path in [VESSELLPATH, SEGMENTPATH]:
    if not os.path.exists(path):
        os.makedirs(path)







all_detections_step1 = pd.DataFrame()
miningpagesdetectfn = build_detectfn(PAGE_MODEL + SAVEDMODEL)
for index, row in pagelist.iterrows():
    print('Page ' + os.path.basename(row['page_path']))

    row, page_imgnp = load_page(row)
    input_tensor = tf.convert_to_tensor(page_imgnp)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = miningpagesdetectfn(input_tensor)
    del page_imgnp
    del input_tensor
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    row['page_detections']= detections
    all_detections_step1 = all_detections_step1.append(row)










Loading model...Done! Took 12.125786066055298 seconds
Page Bonifay_Etudes sur la céramique.pdf0001-042.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-042.png
Page Bonifay_Etudes sur la céramique.pdf0001-041.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-041.png
Page Bonifay_Etudes sur la céramique.pdf0001-044.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-044.png
Page Bonifay_Etudes sur la céramique.pdf0001-045.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-045.png
Page Bonifay_Etudes sur la céramique.pdf0001-043.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-043.png
Page Bonifay_Etudes sur la céramique.pdf0001-076.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-076.png
Page Bonifay_Etudes sur la céramique.pdf0001-040.png
/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la cérami

In [4]:
#all_detections_step2, keylist =unfold_pagedetections(all_detections_step1)
#all_detections_step22 = extract_page_detections(all_detections_step2, keylist, category_index=get_page_labelmap_as_df(PAGE_MODEL + LABELS))
all_detections_step2 = page_detections_toframe(all_detections_step1).drop(columns='page_detections')

page_category_index = get_page_labelmap_as_df(PAGE_MODEL + LABELS)
all_detections_step2 = all_detections_step2.merge(page_category_index, on=['detection_classes'], how='left')

In [5]:
pageids = filter_best_page_detections(all_detections_step2 , lowest_score=0.8)
bestpages = choose_pageid(pageids)
pageid_raw = pd.DataFrame()

#perform ocr page number
for index, row in bestpages.iterrows():
    img = cut_image(row)
    img2 = ocr_pre_processing(img)
    result = pytesseract.image_to_string(img2, config=row['pageid_config'])
    row['newinfo'] = result
    
    pageid_raw = pageid_raw.append(row)
all_detections_step3 = merge_info(all_detections_step2, pageid_raw)
all_detections_step3 = all_detections_step3.apply(ocr_post_processing_pageid, axis=1)

figures = filter_best_vesselprofile_detections(all_detections_step3, lowest_score= 0.7)

['pageid' 'pageinfo']


In [6]:
#detect figure id


figures_step1 = pd.DataFrame()
miningfiguresdetectfn = build_detectfn(FIGID_MODEL + SAVEDMODEL)
for index, row in figures.iterrows():
    print('Figure from ' + os.path.basename(row['page_path']))
    row, figure_imgnp = cut_image_savetemp(row, VESSELLPATH)
    input_tensor = tf.convert_to_tensor(figure_imgnp)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = miningfiguresdetectfn(input_tensor)
    del figure_imgnp
    del input_tensor
    num_detections = int(detections.pop('num_detections'))
    
    detections = {'figid_' + key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    row['figid_detections'] = detections
    detections['figid_num_detections'] = num_detections
    row = extract_detections_figureidv2(row)         
    figures_step1 = figures_step1.append(row)

figid_category_index = get_figid_labelmap_as_df(PAGE_MODEL + LABELS)
figures_step2 = figures_step1.merge(
figid_category_index, on=['figid_detection_classes'], how='left')






Loading model...Done! Took 12.214520454406738 seconds
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pdf0001-042.png
Figure from Bonifay_Etudes sur la céramique.pd

In [7]:
#perform ocr figid
figures_step3 = pd.DataFrame()
for index, row in figures_step2.iterrows():
    print('OCR ' + os.path.basename(row['figure_path']))
    img = cut_image_figid(row)
    img2 = ocr_pre_processing(img)
    row['figid_raw'] = pytesseract.image_to_string(img2, config=row['pageid_config'])
    del img
    del img2
    figures_step3 = figures_step3.append(row)
figures_step3 = figures_step3.apply(ocr_post_processing_figid, axis=1)
figures_step3 = figures_step3.apply(humanreadID, axis=1)
figures_step3 = handleduplicate_humanreadID(figures_step3)

OCR 9f4ea816-274f-4e00-b714-0d0e50a05b27.png
OCR 20086277-bed7-433d-afea-2da5a648c50c.png
OCR 0686063e-9a68-48c0-b942-58e383b9480b.png
OCR e4bfce11-67b7-4e61-a3cf-5605338375cc.png
OCR 6ffd593f-fe75-4dae-ad2a-3545d15c6065.png
OCR 5fdc4863-cd37-43f4-9f50-9b01aed00194.png
OCR dcdd81f4-014c-4a1f-aa5d-87faf01196ab.png
OCR 7804f2c4-13f5-420e-8c9f-146cc2fb68a6.png
OCR 150c765f-ad4c-44a1-8750-3110d60f320d.png
OCR f783fe4d-d96d-4ad1-99b8-8db214833fb0.png
OCR 4d2b6459-bcdb-475f-98b6-d885bfb6c1bc.png
OCR d512dfb2-39e9-4464-be25-04903a365cad.png
OCR 215da195-b5a4-48e1-b5b5-cb2987231280.png
OCR 208a0504-45e9-4789-9396-9ce482e4ef1c.png
OCR 165a83e6-5093-4327-9e95-5b7843f38180.png
OCR c7e1da20-66f9-4d52-9bab-91d94d106a98.png
OCR 6c178d78-025b-4761-a89e-96a57a53ae08.png
OCR 23b71c32-e63e-464a-b83f-010c1fad1411.png
OCR 98d64a32-e21a-4864-ac5f-6908859794ea.png
OCR 94c2eebd-625b-4990-99b7-4e940647b01f.png
OCR 4b0ab7b8-c84c-405c-b7df-6895ecafc6c1.png
OCR 053171f5-f3fc-4e36-88e3-ba70a8b5a697.png
OCR 9f594c

In [8]:

with open(OUTPATH + 'catalogs.jsonl', 'w') as f:
    pubs = figures_step3[['pub_key', 'pub_value']].drop_duplicates()
    pubs.apply(create_catalog_JSONL, file=f, axis=1)
with open(OUTPATH + 'trenches.jsonl', 'w') as f:
    pubs = figures_step3[['pub_key', 'pub_value']].drop_duplicates()
    pubs.apply(create_trench_JSONL, file=f, axis=1)
with open(OUTPATH + 'types.jsonl', 'w') as f:
    figures_step3.apply(create_constructivisttype_JSONL, file=f, axis=1)
with open(OUTPATH + 'types_standalone.jsonl', 'w') as f:
    figures_step3.apply(create_normativtype_JSONL, file=f, axis=1)
with open(OUTPATH + 'finds.jsonl', 'w') as f:
    figures_step3.apply(create_find_JSONL, file=f, axis=1)
with open(OUTPATH + 'drawings.jsonl', 'w') as f:
    figures_step3.apply(create_drawing_JSONL, file=f, axis=1)







figures_step3.to_csv(CSVOUT)
figures_clean = figures_step3[['pub_key','pub_value','figure_tmpid','HRID','detection_scores', 'detection_classesname','page_imgname','pageid_raw','figid_raw','pageid_clean','figid_clean','pageinfo_raw','figure_path','page_path']]
figures_clean.to_csv(CLEANCSVOUT)


#Profile segmentation
#print('Perform image segmentation')
#run_vesselprofile_segmentation(VESSELLPATH, SEGMENTPATH, SEG_MODEL)

In [9]:
shutil.copyfile(PAGE_MODEL + LABELS, OUTPATH +'pages_label_map.pbtxt')

mining_pages_detections = figures_step3.append(bestpages)
mining_pages_detections2 = mining_pages_detections.reindex(axis=0)
pages_grouped = mining_pages_detections2.groupby('pub_name')

for name, group in pages_grouped:
    imgdir = os.path.join(OUTPATH, name + '_pages/')
    if not os.path.exists(imgdir):
        os.makedirs(imgdir)
    Df2TFrecord(group, 'page_path', OUTPATH + name +'_pages.tfrecord')
    for index, row in group.iterrows():
        imgoutpath = imgdir + os.path.basename(row['page_path'])
        print(imgoutpath)
        if not os.path.exists(imgoutpath):
            shutil.copyfile(row['page_path'], imgoutpath)

shutil.copyfile(FIGID_MODEL + LABELS, OUTPATH + 'figures_label_map.pbtxt')
figures_step3 = figures_step3[figures_step3.detection_boxes.notnull()]

figures_grouped = figures_step3.groupby('pub_name')

for name, group in figures_grouped:
    imgdir = os.path.join(OUTPATH, name + '_figures/')
    if not os.path.exists(imgdir):
        os.makedirs(imgdir)
    Df2TFrecord(group, 'figure_path', OUTPATH + name +'_figures.tfrecord')
    for index, row in group.iterrows():
        imgoutpath = imgdir + os.path.basename(str(row['figure_path']))
        print(imgoutpath)
        if not os.path.exists(imgoutpath):
            shutil.copyfile(row['figure_path'], imgoutpath)


[data(filename='/home/images/apply/ZenonID_000147534/Bonifay_Etudes sur la céramique.pdf0001-040.png', object=               classlist                                   detection_boxes  \
56    [pageid, pageinfo]   [0.32662904, 0.11966029, 0.42368525, 0.3036048]   
57    [pageid, pageinfo]  [0.23306902, 0.40703738, 0.31671897, 0.59004384]   
58    [pageid, pageinfo]    [0.3276534, 0.6904425, 0.42215687, 0.87411934]   
59    [pageid, pageinfo]    [0.65040016, 0.4059014, 0.7244852, 0.59790915]   
60    [pageid, pageinfo]      [0.6173767, 0.6849991, 0.7284925, 0.8777505]   
61    [pageid, pageinfo]    [0.76063406, 0.12006601, 0.8538708, 0.3071714]   
62    [pageid, pageinfo]     [0.6505453, 0.105346255, 0.72614, 0.32278103]   
63    [pageid, pageinfo]     [0.7623428, 0.4033566, 0.8356499, 0.59906465]   
64    [pageid, pageinfo]    [0.12260911, 0.6996746, 0.19159614, 0.8622644]   
65    [pageid, pageinfo]     [0.23393486, 0.6971987, 0.3161531, 0.8639428]   
66    [pageid, pageinfo]     [0.